<a href="https://colab.research.google.com/github/dohyun93/hongong_mldl/blob/main/5_2_%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 검증 세트
# 테스트세트를 사용하지 않으면 과대적합인지 과소적합인지 알기가 어렵다.
# 테스트 세트를 사용하지 않고 측정하는 간단한 방법은 훈련세트를 또 나누는것이다. 이 데이터를 검증 세트라고 한다.

# 즉 전체 데이터가 100이라고 했을 때, 훈련세트를 60, 검증세트를 20, 테스트세트를 20으로 구성한다.
# (데이터가 많다면 적은비율로 해도 데이터 대표성을 띄기 때문에 괜찮다.)

import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class'].to_numpy()

print("전체 데이터 shape:", data.shape)
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)
#train_input, train_target 을 훈련세트와 검증 세트로 만든다.
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print("훈련, 검증, 테스트세트 shape:", sub_input.shape, val_input.shape, test_input.shape)
# sub_input: 훈련세트
# val_input: 검증세트

전체 데이터 shape: (6497, 3)
훈련, 검증, 테스트세트 shape: (4157, 3) (1040, 3) (1300, 3)


In [ ]:
 from sklearn.tree import DecisionTreeClassifier
 dt = DecisionTreeClassifier(random_state=42)
 dt.fit(sub_input, sub_target)
 print("1. 훈련세트 스코어: ", dt.score(sub_input, sub_target))
 print("2. 검증세트 스코어: ", dt.score(val_input, val_target))

1. 훈련세트 스코어:  0.9971133028626413
2. 검증세트 스코어:  0.864423076923077


In [ ]:
# 검증 세트를 만드느라 훈련세트가 줄었다.
# 보통 많은 데이터를 훈련에 사용할 수록 좋은 모델이 만들어지는데 그렇다고 검증세트를 너무 조금 떼어내면 검증점수가 들쭉날쭉하고 불안정할 것이다.
# 이럴 때 '교차 검증' (cross-validation) 을 이용하면 안정적인 검증 점수를 얻고 훈련에 더 많은 데이터를 사용할 수 있다.